In [202]:
import cv2 as cv
import numpy as np
import os

Find left ROI

In [203]:
image = cv.imread("image.png")

if image is None:
    print("err no image")
    exit()

Define ROI

In [204]:
x1, y1 = 0, 45
x2, y2 = 23, image.shape[0]  # img.shape[0] = number of rows (img.shape[1] = number of cols)
roi = image[y1:y2, x1:x2]

In [205]:
annotated = image.copy()
cv.rectangle(annotated, (x1, y1), (x2, y2), (0, 0, 255), 2)
cv.imwrite("ann.png", annotated)

True

Find black pixels on roi

In [206]:
# Parametri
prag_negru = 10

# depending on resolution
numar_minim_pixeli_negri = 2

delta_y = 10  # cât să sară în jos la fiecare pas !!

roi_total = roi

# Offset inițial relativ în ROI
current_y = 0
ys = []

while current_y < roi_total.shape[0]:
    roi_partial = roi_total[current_y:, :]  

    y_offset = None
    for y in range(roi_partial.shape[0]):
        linie = roi_partial[y, :]
        numar_negru = np.sum(linie < prag_negru)
        if numar_negru >= numar_minim_pixeli_negri:
            y_offset = y
            break

    if y_offset is None:
        break  # nu s-a mai găsit nimic, ieșim

    real_y = y1 + current_y + y_offset
    ys.append(real_y)
    cv.line(annotated, (x1, real_y), (x2, real_y), (0, 255, 0), 2)

    # Actualizăm punctul de plecare pentru următoarea căutare
    current_y += y_offset + delta_y
    cv.line(annotated, (x1, real_y), (x2, real_y), (0, 255, 0), 2)

In [207]:
cv.imwrite("ann.png", annotated)
os.makedirs("parts", exist_ok=True)

# play with them, find yourself :)
padding_up = 6
padding_down = 2

# Crop și salvare între perechi succesive de y
for i in range(len(ys) - 1):
    y_start = ys[i] - padding_up
    y_end = ys[i + 1] - padding_down
    if y_end > y_start:
        bucata = image[y_start:y_end, 0:image.shape[1]]
        cv.imwrite(f"parts/part_{i:02d}.png", bucata)